# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Initialize-Environment" data-toc-modified-id="Initialize-Environment-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Initialize Environment</a></div><div class="lev1 toc-item"><a href="#Gather-Data" data-toc-modified-id="Gather-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Gather Data</a></div><div class="lev1 toc-item"><a href="#Relationship-between-controllability-and-topology" data-toc-modified-id="Relationship-between-controllability-and-topology-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Relationship between controllability and topology</a></div><div class="lev1 toc-item"><a href="#Compare-Controllability-Distributions" data-toc-modified-id="Compare-Controllability-Distributions-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Compare Controllability Distributions</a></div>

# Initialize Environment

In [ ]:
try:
    %load_ext autoreload
    %autoreload 2
    %reset
except:
    print 'NOT IPYTHON'

from __future__ import division

import os
import sys
import glob
import time

import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import scipy.io as io
import h5py
import pickle as pkl
import matplotlib.pyplot as plt
from matplotlib import rcParams

sys.path.append('/Users/akhambhati/Developer/hoth_research/Echobase')
import Echobase
convert_adj_matr_to_cfg_matr = Echobase.Network.Transforms.configuration.convert_adj_matr_to_cfg_matr
convert_conn_vec_to_adj_matr = Echobase.Network.Transforms.configuration.convert_conn_vec_to_adj_matr

rcParams = Echobase.Plotting.fig_format.update_rcparams(rcParams)
rcParams.update(rcParams)

path_AtlasData = '/Users/akhambhati/Remotes/CORE.MRI_Atlases'
path_CoreData = '/Users/akhambhati/Remotes/CORE.RAM_Stim'
path_PeriphData = '/Users/akhambhati/Remotes/RSRCH.RAM_Stim'
#path_InpData = path_PeriphData + '/e01-FuncNetw'
#path_InpData_Baseline = path_PeriphData + '/e01b-FuncNetw'
path_ExpData = path_PeriphData + '/e04-StructControl'

for path in [path_CoreData, path_PeriphData, path_ExpData]:
    if not os.path.exists(path):
        print('Path: {}, does not exist'.format(path))
        os.makedirs(path)

# Gather Data

In [ ]:
data_path = glob.glob('{}/Controllability/*/*/*/*'.format(path_CoreData))

subj_dict = {}
for pth in data_path:
    name_list = pth.split('/')[-4:]
    
    subj_type = name_list[0]
    subj_id = name_list[2]    
    atlas_type = name_list[1]
    if atlas_type == 'lausanne':
        trk_type = name_list[3].split('.')[-4]
        scale_type = name_list[3].split('.')[-5]
        if scale_type != 'ROIv_scale125_dilated':
            continue
        adj_path = glob.glob('{}/Adjacency_Matrices/{}/{}/{}/*.{}.{}.*'.format(path_CoreData, subj_type, atlas_type, subj_id, scale_type, trk_type))[0]
    else:
        trk_type = name_list[3].split('.')[-4]
        adj_path = glob.glob('{}/Adjacency_Matrices/{}/{}/{}/*.{}.*'.format(path_CoreData, subj_type, atlas_type, subj_id, trk_type))[0]        

    if not atlas_type in subj_dict.keys():
        subj_dict[atlas_type] = {}
    
    if not trk_type in subj_dict[atlas_type].keys():
        subj_dict[atlas_type][trk_type] = {}
        
    if not subj_type in subj_dict[atlas_type][trk_type].keys():
        subj_dict[atlas_type][trk_type][subj_type] = {}
    
    if not subj_id in subj_dict[atlas_type][trk_type][subj_type].keys():
        subj_dict[atlas_type][trk_type][subj_type][subj_id] = {
            'control_path': pth,
            'adj_path': adj_path}  

# Relationship between controllability and topology

In [ ]:
%matplotlib inline
#excl_subj = ['C013', 'C010', 'C015', 'C006', 'C023']
excl_subj = []
data_table = {'Atlas_Type': [],
              'Track_Type': [],
              'Subj_Type': [],
              'Subj_ID': [],
              'ROI_ID': [],
              'Avg_Control': [],
              'Mod_Control': [],
              'Node_Strength': []}

for atlas_type in subj_dict.keys():
    for trk_type in subj_dict[atlas_type].keys():
        for subj_type in subj_dict[atlas_type][trk_type].keys():
            for subj_id in subj_dict[atlas_type][trk_type][subj_type].keys():
                if subj_id in excl_subj:
                    continue
                adj_pth = subj_dict[atlas_type][trk_type][subj_type][subj_id]['adj_path']                
                ctl_pth = subj_dict[atlas_type][trk_type][subj_type][subj_id]['control_path']
                
                # Get connectivity
                df_adj = io.loadmat(adj_pth)
                node_str = df_adj['connectivity'].mean(axis=0)
                
                df_ctl = io.loadmat(ctl_pth)
                avg_control = df_ctl['avg_vector'][:, 0]
                mod_control = df_ctl['modal_vector'][:, 0]
                
                for nd_i, (ac, mc, ns) in enumerate(zip(avg_control, mod_control, node_str)):  
                    data_table['Atlas_Type'].append(atlas_type)
                    data_table['Track_Type'].append(trk_type)
                    data_table['Subj_Type'].append(subj_type)
                    data_table['Subj_ID'].append(subj_id)
                    data_table['ROI_ID'].append(nd_i)
                    data_table['Avg_Control'].append(ac)
                    data_table['Mod_Control'].append(mc)
                    data_table['Node_Strength'].append(ns)                    

df = pd.DataFrame(data_table, columns=data_table.keys())

sel_df = df[df['Atlas_Type'] == 'lausanne']
sel_df = sel_df[sel_df['Track_Type'] == 'qa']

control_ns = sel_df[sel_df['Subj_Type'] == 'Control_Subjects']['Node_Strength']
control_ac = sel_df[sel_df['Subj_Type'] == 'Control_Subjects']['Avg_Control']
control_mc = sel_df[sel_df['Subj_Type'] == 'Control_Subjects']['Mod_Control']
epileps_ns = sel_df[sel_df['Subj_Type'] == 'Epilepsy_Subjects']['Node_Strength']
epileps_ac = sel_df[sel_df['Subj_Type'] == 'Epilepsy_Subjects']['Avg_Control']
epileps_mc = sel_df[sel_df['Subj_Type'] == 'Epilepsy_Subjects']['Mod_Control']


plt.figure()
ax = plt.subplot(111)
ax.scatter(control_ns, control_ac, lw=0, s=1.0, color='b')
ax.scatter(epileps_ns, epileps_ac, lw=0, s=1.0, color='r')
ax.xaxis.set_ticks_position('bottom')
ax.yaxis.set_ticks_position('left')
ax.set_xlabel('Node Strength')
ax.set_ylabel('Average Controllability')
plt.savefig('./e04-Figures/Topology.NS_AC.png')


plt.figure()
ax = plt.subplot(111)
ax.scatter(control_ns, control_mc, lw=0, s=1.0, color='b')
ax.scatter(epileps_ns, epileps_mc, lw=0, s=1.0, color='r')
ax.xaxis.set_ticks_position('bottom')
ax.yaxis.set_ticks_position('left')
ax.set_xlabel('Node Strength')
ax.set_ylabel('Modal Controllability')
plt.savefig('./e04-Figures/Topology.NS_MC.png')

print('--- Control Subjects ---')
print('NS vs AC: {}'.format(stats.pearsonr(control_ns, control_ac)))
print('NS vs MC: {}'.format(stats.pearsonr(control_ns, control_mc)))

print('--- Epilepsy Subjects ---')
print('NS vs AC: {}'.format(stats.pearsonr(epileps_ns, epileps_ac)))
print('NS vs MC: {}'.format(stats.pearsonr(epileps_ns, epileps_mc)))

# Compare Controllability Distributions

In [ ]:
%matplotlib inline

sel_df = df[df['Atlas_Type'] == 'lausanne']
sel_df = sel_df[sel_df['Track_Type'] == 'gfa']

grp_df = sel_df.groupby([sel_df['ROI_ID'], sel_df['Subj_Type']])
pop_mean = grp_df.aggregate(lambda x: np.mean(x)).unstack()
pop_ste = grp_df.aggregate(lambda x: np.std(x) / np.sqrt(x.count())).unstack()
epileps_ord_ac_ix = np.argsort(pop_mean['Avg_Control']['Epilepsy_Subjects'])
epileps_ord_mc_ix = np.argsort(pop_mean['Mod_Control']['Epilepsy_Subjects'])

### Average Control
plt.figure()
ax = plt.subplot(111)
ax.errorbar(np.arange(len(epileps_ord_ac_ix)),
            np.array(pop_mean['Avg_Control']['Epilepsy_Subjects'])[epileps_ord_ac_ix],
            yerr=np.array(pop_ste['Avg_Control']['Epilepsy_Subjects'])[epileps_ord_ac_ix])

ax.errorbar(np.arange(len(epileps_ord_ac_ix))+0.2,
            np.array(pop_mean['Avg_Control']['Control_Subjects'])[epileps_ord_ac_ix],
            yerr=np.array(pop_ste['Avg_Control']['Control_Subjects'])[epileps_ord_ac_ix])

# Test for difference
sig_diff_ix = []
alpha = 0.05 / len(epileps_ord_ac_ix)
for ii, ix in enumerate(epileps_ord_ac_ix):
    sel_roi = sel_df[sel_df['ROI_ID'] == ix]
    sel_control = sel_roi[sel_roi['Subj_Type'] == 'Control_Subjects']
    sel_epileps = sel_roi[sel_roi['Subj_Type'] == 'Epilepsy_Subjects']
    
    if stats.ttest_ind(sel_epileps['Avg_Control'], sel_control['Avg_Control'])[1] < alpha:
        ax.scatter(ii, np.array(pop_mean['Avg_Control']['Epilepsy_Subjects'])[ix]*1.001, marker='x', color='r')
        sig_diff_ix.append(ix)

ax.set_xlim([-1, len(epileps_ord_ac_ix)])
ax.xaxis.set_ticks_position('bottom')
ax.yaxis.set_ticks_position('left')
ax.set_xlabel('Brain Regions')
ax.set_ylabel('Average Controllability')
ax.legend(['RAM Subjects', 'Healthy Controls'])


### Modal Control
plt.figure()
ax = plt.subplot(111)
ax.errorbar(np.arange(len(epileps_ord_mc_ix)),
            np.array(pop_mean['Mod_Control']['Epilepsy_Subjects'])[epileps_ord_mc_ix],
            yerr=np.array(pop_ste['Mod_Control']['Epilepsy_Subjects'])[epileps_ord_mc_ix])

ax.errorbar(np.arange(len(epileps_ord_mc_ix))+0.25,
            np.array(pop_mean['Mod_Control']['Control_Subjects'])[epileps_ord_mc_ix],
            yerr=np.array(pop_ste['Mod_Control']['Control_Subjects'])[epileps_ord_mc_ix])

# Test for difference
sig_diff_ix = []
alpha = 0.05 / len(epileps_ord_mc_ix)
for ii, ix in enumerate(epileps_ord_mc_ix):
    sel_roi = sel_df[sel_df['ROI_ID'] == ix]
    sel_control = sel_roi[sel_roi['Subj_Type'] == 'Control_Subjects']
    sel_epileps = sel_roi[sel_roi['Subj_Type'] == 'Epilepsy_Subjects']
    
    if stats.ttest_ind(sel_epileps['Mod_Control'], sel_control['Mod_Control'])[1] < alpha:
        ax.scatter(ii, np.array(pop_mean['Mod_Control']['Epilepsy_Subjects'])[ix]*1.001, marker='x', color='r')
        sig_diff_ix.append(ix)

ax.set_xlim([-1, len(epileps_ord_mc_ix)])
ax.xaxis.set_ticks_position('bottom')
ax.yaxis.set_ticks_position('left')
ax.set_xlabel('Brain Regions')
ax.set_ylabel('Modal Controllability')
ax.legend(['RAM Subjects', 'Healthy Controls'])